In [91]:
import json
import numpy as np
import pandas as pd
import requests
import re
import datetime

In [109]:
repos_list = pd.read_csv('repos.list.txt', header=None, names=["repo"])

In [93]:
def url_list(repos_list):
    urls = []
    for i in range(len(repos_list)):
        url = 'https://api.github.com/repos/{}/commits?per_page=100'.format(repos_list.iloc[i, 0])
        urls = urls + [url]
        
    return urls

In [94]:
ACCESS_TOKEN = 'b0e0127248f5a84b599800260dcb824779ac04b1'
params = {
    'access_token': ACCESS_TOKEN
}


In [95]:
# Count the number of total pages, req = requests.get(...)
def read_page_count(req):
    # Get the commit-specific link header
    git_header = req.headers['Link'].split(',')
    
    # Split the header into individual lines and select the reference to last page
    ref = [bool(re.search('rel="last"', i)) for i in git_header]
    last_ref = git_header[ref.index(True)]
    
    # Extract last page number and convert to int
    page_count = int(re.search('[&?]page=(.+?)>;', last_ref).group(1))
    
    return page_count
    

In [101]:
# Load the data from one page, url = '...?per_page=30'
def data_from_one_page(page, url):
    # Concat the url query for specific page
    query = url + '&page=' + str(page)
    binary_data = requests.get(url=query, params=params).content
    data_list_this_page = json.loads(binary_data)
    
    sha = []
    author = []
    author_email = []
    author_timestamp = []
    committer = []
    committer_email = []
    committer_timestamp = []
    subject = []
    message = []
    for item in data_list_this_page:
        # Hash
        sha = sha + [item['sha']]
        # Commits_df
        author = author + [item['commit']['author']['name']]
        author_email = author_email + [item['commit']['author']['email']]
        author_timestamp = author_timestamp + [item['commit']['author']['date']]
        committer = committer + [item['commit']['committer']['name']]
        committer_email = committer_email + [item['commit']['committer']['email']]
        committer_timestamp = committer_timestamp + [item['commit']['committer']['date']]
        # Messages_df
        message_item = item['commit']['message'].split('\n', 1)
        subject = subject + [message_item[0]]
        message = message + [item['commit']['message']]
        
    commits_df = pd.DataFrame({'hash': sha, 
                               'author': author, 'author email': author_email, 'author timestamp': author_timestamp, 
                              'committer': committer, 'committer email': committer_email, 'committer timestamp': committer_timestamp})
    messages_df = pd.DataFrame({'hash': sha, 'subject': subject, 'message': message})
    
    return commits_df, messages_df
    

In [102]:
def data_from_one_repo(url):
    # Find the number of pages of commits
    req = requests.get(url=url, params=params)
    pages = read_page_count(req)
    commits_df = pd.DataFrame({'hash': [], 
                              'author': [], 'author email': [], 'author timestamp': [], 
                              'committer': [], 'committer email': [], 'committer timestamp': []})
    messages_df = pd.DataFrame({'hash': [], 'subject': [], 'message': []})
    
    for page in range(1, pages + 1):
        print('Now collecting data from page ' + str(page) + '...')
        one_page_commits_df, one_page_messages_df = data_from_one_page(page, url)
        commits_df = commits_df.append(one_page_commits_df)
        messages_df = messages_df.append(one_page_messages_df)
        
    return commits_df, messages_df
        

In [103]:
def data_from_all_repos(repos_list):
    urls = url_list(repos_list)
    commits_df = pd.DataFrame({'hash': [], 
                              'author': [], 'author email': [], 'author timestamp': [], 
                              'committer': [], 'committer email': [], 'committer timestamp': []})
    messages_df = pd.DataFrame({'hash': [], 'subject': [], 'message': []})
    k = 1
    
    for url in urls:
        print('Now collecting data from the ' + str(k) + 'th url: ' + url + " ... ")
        one_repo_commits_df, one_repo_messages_df = data_from_one_repo(url)
        commits_df = commits_df.append(one_repo_commits_df)
        messages_df = messages_df.append(one_repo_messages_df)
        k += 1
        
    return commits_df, messages_df
        

In [89]:
# Find the size of the output files
def get_FileSize(filePath):
    fsize = os.path.getsize(filePath)
    fsize = fsize/float(1024*1024)
    return round(fsize,2)

In [104]:
begin = datetime.datetime.now()
co_ex, me_ex = data_from_all_repos(repos_list)
end = datetime.datetime.now()

Now collecting data from the 1th url: https://api.github.com/repos/freeCodeCamp/freeCodeCamp/commits?per_page=100 ... 
Now collecting data from page 1...
Now collecting data from page 2...
Now collecting data from page 3...
Now collecting data from page 4...
Now collecting data from page 5...
Now collecting data from page 6...
Now collecting data from page 7...
Now collecting data from page 8...
Now collecting data from page 9...
Now collecting data from page 10...
Now collecting data from page 11...
Now collecting data from page 12...
Now collecting data from page 13...
Now collecting data from page 14...
Now collecting data from page 15...
Now collecting data from page 16...
Now collecting data from page 17...
Now collecting data from page 18...
Now collecting data from page 19...
Now collecting data from page 20...
Now collecting data from page 21...
Now collecting data from page 22...
Now collecting data from page 23...
Now collecting data from page 24...
Now collecting data from p

Now collecting data from page 12...
Now collecting data from page 13...
Now collecting data from page 14...
Now collecting data from page 15...
Now collecting data from page 16...
Now collecting data from page 17...
Now collecting data from page 18...
Now collecting data from page 19...
Now collecting data from page 20...
Now collecting data from page 21...
Now collecting data from page 22...
Now collecting data from page 23...
Now collecting data from page 24...
Now collecting data from page 25...
Now collecting data from page 26...
Now collecting data from page 27...
Now collecting data from page 28...
Now collecting data from page 29...
Now collecting data from page 30...
Now collecting data from page 31...
Now collecting data from page 32...
Now collecting data from page 33...
Now collecting data from page 34...
Now collecting data from page 35...
Now collecting data from page 36...
Now collecting data from page 37...
Now collecting data from page 38...
Now collecting data from pag

Now collecting data from page 19...
Now collecting data from page 20...
Now collecting data from page 21...
Now collecting data from page 22...
Now collecting data from page 23...
Now collecting data from page 24...
Now collecting data from page 25...
Now collecting data from page 26...
Now collecting data from page 27...
Now collecting data from page 28...
Now collecting data from page 29...
Now collecting data from page 30...
Now collecting data from page 31...
Now collecting data from page 32...
Now collecting data from page 33...
Now collecting data from page 34...
Now collecting data from page 35...
Now collecting data from page 36...
Now collecting data from page 37...
Now collecting data from page 38...
Now collecting data from page 39...
Now collecting data from page 40...
Now collecting data from page 41...
Now collecting data from page 42...
Now collecting data from page 43...
Now collecting data from page 44...
Now collecting data from page 45...
Now collecting data from pag

Now collecting data from page 137...
Now collecting data from page 138...
Now collecting data from page 139...
Now collecting data from page 140...
Now collecting data from page 141...
Now collecting data from page 142...
Now collecting data from page 143...
Now collecting data from page 144...
Now collecting data from page 145...
Now collecting data from page 146...
Now collecting data from page 147...
Now collecting data from page 148...
Now collecting data from page 149...
Now collecting data from page 150...
Now collecting data from page 151...
Now collecting data from page 152...
Now collecting data from page 153...
Now collecting data from page 154...
Now collecting data from page 155...
Now collecting data from page 156...
Now collecting data from page 157...
Now collecting data from page 158...
Now collecting data from page 159...
Now collecting data from page 160...
Now collecting data from page 161...
Now collecting data from page 162...
Now collecting data from page 163...
N

Now collecting data from page 359...
Now collecting data from page 360...
Now collecting data from page 361...
Now collecting data from page 362...
Now collecting data from page 363...
Now collecting data from page 364...
Now collecting data from page 365...
Now collecting data from page 366...
Now collecting data from page 367...
Now collecting data from page 368...
Now collecting data from page 369...
Now collecting data from page 370...
Now collecting data from page 371...
Now collecting data from page 372...
Now collecting data from page 373...
Now collecting data from page 374...
Now collecting data from page 375...
Now collecting data from page 376...
Now collecting data from page 377...
Now collecting data from page 378...
Now collecting data from page 379...
Now collecting data from page 380...
Now collecting data from page 381...
Now collecting data from page 382...
Now collecting data from page 383...
Now collecting data from page 384...
Now collecting data from page 385...
N

...

Now collecting data from page 3858...
Now collecting data from page 3859...
Now collecting data from page 3860...
Now collecting data from page 3861...
Now collecting data from page 3862...
Now collecting data from page 3863...
Now collecting data from page 3864...
Now collecting data from page 3865...
Now collecting data from page 3866...
Now collecting data from page 3867...
Now collecting data from page 3868...
Now collecting data from page 3869...
Now collecting data from page 3870...
Now collecting data from page 3871...
Now collecting data from page 3872...
Now collecting data from page 3873...
Now collecting data from page 3874...
Now collecting data from page 3875...
Now collecting data from page 3876...
Now collecting data from page 3877...
Now collecting data from page 3878...
Now collecting data from page 3879...
Now collecting data from page 3880...
Now collecting data from page 3881...
Now collecting data from page 3882...
Now collecting data from page 3883...
Now collecti

Now collecting data from page 4074...
Now collecting data from page 4075...
Now collecting data from page 4076...
Now collecting data from page 4077...
Now collecting data from page 4078...
Now collecting data from page 4079...
Now collecting data from page 4080...
Now collecting data from page 4081...
Now collecting data from page 4082...
Now collecting data from page 4083...
Now collecting data from page 4084...
Now collecting data from page 4085...
Now collecting data from page 4086...
Now collecting data from page 4087...
Now collecting data from page 4088...
Now collecting data from page 4089...
Now collecting data from page 4090...
Now collecting data from page 4091...
Now collecting data from page 4092...
Now collecting data from page 4093...
Now collecting data from page 4094...
Now collecting data from page 4095...
Now collecting data from page 4096...
Now collecting data from page 4097...
Now collecting data from page 4098...
Now collecting data from page 4099...
Now collecti

Now collecting data from page 4290...
Now collecting data from page 4291...
Now collecting data from page 4292...
Now collecting data from page 4293...
Now collecting data from page 4294...
Now collecting data from page 4295...
Now collecting data from page 4296...
Now collecting data from page 4297...
Now collecting data from page 4298...
Now collecting data from page 4299...
Now collecting data from page 4300...
Now collecting data from page 4301...
Now collecting data from page 4302...
Now collecting data from page 4303...
Now collecting data from page 4304...
Now collecting data from page 4305...
Now collecting data from page 4306...
Now collecting data from page 4307...
Now collecting data from page 4308...
Now collecting data from page 4309...
Now collecting data from page 4310...
Now collecting data from page 4311...
Now collecting data from page 4312...
Now collecting data from page 4313...
Now collecting data from page 4314...
Now collecting data from page 4315...
Now collecti

Now collecting data from page 4506...
Now collecting data from page 4507...
Now collecting data from page 4508...
Now collecting data from page 4509...
Now collecting data from page 4510...
Now collecting data from page 4511...
Now collecting data from page 4512...
Now collecting data from page 4513...
Now collecting data from page 4514...
Now collecting data from page 4515...
Now collecting data from page 4516...
Now collecting data from page 4517...
Now collecting data from page 4518...
Now collecting data from page 4519...
Now collecting data from page 4520...
Now collecting data from page 4521...
Now collecting data from page 4522...
Now collecting data from page 4523...
Now collecting data from page 4524...
Now collecting data from page 4525...
Now collecting data from page 4526...
Now collecting data from page 4527...
Now collecting data from page 4528...
Now collecting data from page 4529...
Now collecting data from page 4530...
Now collecting data from page 4531...
Now collecti

JSONDecodeError: Expecting value: line 1 column 1 (char 0)